In [16]:
import pandas as pd
import numpy as np
import glob
import xlrd
import re
import pprint
import pyreadstat
import pingouin as pg
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [4]:
dfi, meta = pyreadstat.read_sav('C:/Users/IOR_C/Downloads/Base de datos FGF21 final.sav')

In [5]:
dfi.head()

,Folio,Iniciales,Base,Folio_BaseOriginal,KDOQI_PAREADO,Registro,Telefono,Edad,Sexo,Numero_de_cigarrillos,...,FGF21orina24h,Leptina,Adiponectina,Microalbuminuria_seleccion,Radio_LDLox_LDL,Enfermedad_renal_cronica,Albuminuria_si_o_no,Obesidad,filter_$,LNFGF21
0,1.0,NaN,0.0,1.0,NaN,207740.0,5.524016e+09,51.0,0.0,0.0,...,NaN,38.9,8.2,0.0,0.600000,1.0,1.0,1.0,1.0,6.076348
1,2.0,NaN,0.0,2.0,NaN,159302.0,5.534609e+09,63.0,1.0,0.0,...,NaN,4.3,10.4,1.0,0.345745,1.0,0.0,0.0,1.0,4.773579
2,3.0,NaN,0.0,3.0,NaN,255596.0,5.537292e+09,57.0,0.0,0.0,...,NaN,10.8,5.6,0.0,0.505319,1.0,1.0,0.0,1.0,6.402231
3,4.0,NaN,0.0,4.0,NaN,211412.0,8.284204e+07,52.0,1.0,0.0,...,NaN,2.4,59.2,0.0,0.419048,0.0,1.0,0.0,0.0,5.357977
4,5.0,NaN,0.0,5.0,NaN,248321.0,5.523499e+09,38.0,0.0,0.0,...,NaN,19.4,8.1,0.0,0.549333,1.0,1.0,1.0,1.0,5.959406


In [10]:
corr = pg.pairwise_corr(dfi, columns=['FGF21_OFICIAL', "CCint", 'IMC', 'TGbasal', 'HDL','LDL', 'GFR_1', 'ACTANTIOX_OFICIAL',
    'LDL_OFICIAL', 'Microalbuminuria_24h','HbA1c'], method='spearman', padjust='bonf')
pd.DataFrame(corr).to_csv('C:/Users/IOR_C/Downloads/corr_fgf21_pairwise.csv', index=False)

In [12]:
corr = dfi[['FGF21_OFICIAL', "CCint", 'IMC', 'TGbasal', 'HDL','LDL', 'GFR_1', 'ACTANTIOX_OFICIAL',
    'LDL_OFICIAL', 'Microalbuminuria_24h','HbA1c']].rcorr()
pd.DataFrame(corr).to_csv('C:/Users/IOR_C/Downloads/corr_fgf21_matrix.csv')

In [26]:
dfr = dfi.dropna(subset=['GFR_1','ACTANTIOX_OFICIAL'])
X = dfr['GFR_1']
y = dfr['ACTANTIOX_OFICIAL']
X = sm.add_constant(X)

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      ACTANTIOX_OFICIAL   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     12.37
Date:                Thu, 09 Apr 2020   Prob (F-statistic):           0.000489
Time:                        21:53:22   Log-Likelihood:                -2505.4
No. Observations:                 380   AIC:                             5015.
Df Residuals:                     378   BIC:                             5023.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        574.9265     23.253     24.724      0.000     529.204     620.649
GFR_1         -0.9624      0.274     -3.517      0.000      -1.500      -0.424
==============================================================================
Omnibus:                       91.896   Durbin-Watson:                   1.408
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              191.722
Skew:                           1.261   Prob(JB):                     2.33e-42
Kurtosis:                       5.397   Cond. No.                         217.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
# Note the difference in argument order
model = smf.ols(formula='ACTANTIOX_OFICIAL~GFR_1',data =dfr).fit()
# Print out the statistics
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      ACTANTIOX_OFICIAL   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     12.37
Date:                Thu, 09 Apr 2020   Prob (F-statistic):           0.000489
Time:                        21:55:21   Log-Likelihood:                -2505.4
No. Observations:                 380   AIC:                             5015.
Df Residuals:                     378   BIC:                             5023.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    574.9265     23.253     24.724      0.0

In [28]:
# Note the difference in argument order
model = smf.ols(formula='GFR_1~ACTANTIOX_OFICIAL',data =dfr).fit()
# Print out the statistics
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  GFR_1   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     12.37
Date:                Thu, 09 Apr 2020   Prob (F-statistic):           0.000489
Time:                        21:56:50   Log-Likelihood:                -1864.2
No. Observations:                 380   AIC:                             3732.
Df Residuals:                     378   BIC:                             3740.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            94.6827      4.97